In [ ]:
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/subject05.npz &> /dev/null

In [ ]:
import MRzeroCore as mr0
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi
import torch

(flash_dwi)=
# DWI preparation with FLASH readout

Change the b-value in build_seq() to the desired value

In [ ]:
def build_seq(fov=0.2) -> mr0.Sequence:
    seq = mr0.Sequence()

    # Needed, otherwise prep produces stronger first readout
    dummies = 1

    # Add DWI with a 90-180-90 prep sequence
    b = 1000  # s / mm^2
    t_grad = 10e-3
    t_rf = 2e-3
    # We manually scale xy grads for FOV and leave z alone.
    # Convert mm to m -> 1e3
    k = 1e3 * np.sqrt(b / (2/3 * t_grad + t_rf))
    print(f"b-value: {b} -> gradient moment: {k}")

    rep = seq.new_rep(2)
    rep.pulse.angle = pi/2
    rep.pulse.usage = mr0.PulseUsage.EXCIT
    rep.event_time[0] = t_rf
    rep.event_time[1] = t_grad
    rep.gradm[1, 2] = k

    rep = seq.new_rep(2)
    rep.pulse.angle = pi
    rep.pulse.usage = mr0.PulseUsage.REFOC
    rep.event_time[0] = t_rf
    rep.event_time[1] = t_grad
    rep.gradm[1, 2] = k
    rep.gradm[1, 0] = -96

    rep = seq.new_rep(2)
    rep.pulse.angle = pi/2
    rep.pulse.usage = mr0.PulseUsage.STORE
    rep.event_time[0] = t_rf
    rep.event_time[1] = 10e-3  # Could increase this for more T1 weighting

    # FLASH readout
    for i in range(64 + dummies):
        rep = seq.new_rep(2 + 64 + 1)
        rep.pulse.usage = mr0.PulseUsage.EXCIT
        rep.pulse.angle = 7 * pi/180
        rep.pulse.phase = 0.5 * 117 * (i**2+i+2) * pi / 180

        rep.event_time[0] = 2e-3  # Pulse
        rep.event_time[1] = 2e-3  # Rewinder
        rep.event_time[2:-1] = 0.08e-3  # Readout
        rep.event_time[-1] = 2e-3  # Spoiler

        rep.gradm[1, 0] = 96 - 33
        rep.gradm[2:-1, 0] = 1
        rep.gradm[-1, 0] = 96 - 31

        if i >= dummies:
            # Linear reordered phase encoding
            rep.gradm[1, 1] = i - 32
            rep.gradm[-1, 1] = -rep.gradm[1, 1]

            rep.adc_usage[2:-1] = 1
            rep.adc_phase[2:-1] = pi - rep.pulse.phase

    seq.normalized_grads = False
    for rep in seq:
        # Don't scale z-gradients used for diffusion
        rep.gradm[:, :2] /= fov

    return seq

In [ ]:
# Simulate
phantom = mr0.VoxelGridPhantom.brainweb("subject05.npz")
phantom = phantom.interpolate(64, 64, 32).slices([16])
data = phantom.build()

seq = build_seq(fov=data.size[0])
seq.plot_kspace_trajectory()

graph = mr0.compute_graph(seq, data)
signal = mr0.execute_graph(graph, seq, data, print_progress=False)

In [ ]:
# Plot the result
kspace = signal.view(64, 64)
reco = torch.fft.fftshift(
    torch.fft.fft2(torch.fft.fftshift(kspace), norm="forward")
)

plt.figure(figsize=(8, 7))
plt.subplot(221)
plt.title("abs(reco)")
plt.imshow(reco.abs().cpu().flip(0), vmin=0)
plt.colorbar()
plt.axis("off")
plt.subplot(222)
plt.title("angle(reco)")
plt.imshow(reco.angle().cpu().flip(0), vmin=-np.pi, vmax=np.pi, cmap="twilight")
plt.colorbar()
plt.axis("off")
plt.subplot(223)
plt.title("log(abs(kspace))")
plt.imshow(kspace.abs().log().cpu().flip(0))
plt.colorbar()
plt.axis("off")
plt.subplot(224)
plt.title("Proton Density")
plt.imshow(phantom.PD.cpu()[:, :, 0].T, origin='lower', vmin=0, vmax=1)
plt.colorbar()
plt.axis("off")
plt.show()